In [6]:
import numpy as np
import pandas as pd
import sqlite3

con = sqlite3.connect('data/nflPPdb.sqlite')

In [107]:
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.widgets import Panel, Tabs

#Create data frames for RBs and WRs
df1 = pd.read_sql_query('SELECT combine.name, combine.fortyyd, combine.year FROM combine\
                        WHERE combine.year < 2009 AND combine.pickround != 0', con)
df2 = pd.read_sql_query('SELECT combine.name, combine.year, players.position FROM combine, players\
                        WHERE combine.name = players.name AND combine.year = players.draft_year', con)
df3 = pd.merge(df1, df2, on=['name', 'year'], how='inner', suffixes=('df1','df2'))
df3 = df3.drop_duplicates(subset='name', keep=False)
df4 = pd.read_sql_query('SELECT DISTINCT combine.name, rr.rushing_yards, rr.receiving_yards\
                        FROM combine, rr WHERE combine.name = rr.name AND combine.year < 2009', con)
df4 = pd.pivot_table(df4,index=['name'],aggfunc=np.sum).reset_index().fillna(0)
df4['totYds'] = (df4['receiving_yards'] + df4['rushing_yards']).astype(int)
df5 = pd.merge(df3,df4, on='name', how='inner', suffixes=('df3','df4'))
df6 = df5[df5.position == 'RB']
df6 = df6[df6.fortyyd < 5] #remove outliers
df7 = df5[df5.position == 'WR']
df7 = df7[df7.fortyyd < 5] #remove outliers

#Create QB data frame
df8 = pd.read_sql_query('SELECT DISTINCT combine.name, combine.fortyyd, passing.passing_yards FROM combine, passing\
                        WHERE combine.name = passing.name AND combine.year < 2009', con)
df8['count'] = 1 #use to get 40 yard time back after aggregating
df8 = pd.pivot_table(df8,index=['name'],aggfunc=np.sum).reset_index()
df8['fortyyd'] = df8['fortyyd']/df8['count']
df8 = df8[df8.passing_yards > 175] #remove outliers

output_notebook()
#output_file('40yd.html')

#RB plot
source = ColumnDataSource(data=dict(x=df6['fortyyd'], y=df6['totYds'], rush=df6['rushing_yards'], 
                                    rec=df6['receiving_yards'], name=df6['name'],))
hover = HoverTool(tooltips=[('Player', '@name'),('40 Yard Dash Time', '$x{1.11}'),('Career Rushing Yards', '@rush'),
                  ('Career Receiving Yards', '@rec'), ('Total Yards', '@y'),])

p1 = figure(plot_width=600, plot_height=600, tools="pan,wheel_zoom,box_zoom,reset,resize", 
           title="RB: Total Yards by 40 Yard Dash",
           x_axis_label ="40 Yard Dash Time", y_axis_label ="Career Rushing and Receiving Yards")
p1.add_tools(hover)
p1.circle('x', 'y', size=7, color='cyan', source=source)
tab1 = Panel(child=p1, title="RB")

#WR plot
source = ColumnDataSource(data=dict(x=df7['fortyyd'], y=df7['totYds'], rush=df7['rushing_yards'], 
                                    rec=df7['receiving_yards'], name=df7['name'],))
hover = HoverTool(tooltips=[('Player', '@name'),('40 Yard Dash Time', '$x{1.11}'),('Career Rushing Yards', '@rush'),
                  ('Career Receiving Yards', '@rec'), ('Total Yards', '@y'),])

p2 = figure(plot_width=600, plot_height=600, tools="pan,wheel_zoom,box_zoom,reset,resize", 
           title="WR: Total Yards by 40 Yard Dash",
           x_axis_label ="40 Yard Dash Time", y_axis_label ="Career Rushing and Receiving Yards")
p2.add_tools(hover)
p2.circle('x', 'y', size=7, color='cyan', source=source)
tab2 = Panel(child=p2, title="WR")

#QB plot
source = ColumnDataSource(data=dict(x=df8['fortyyd'], y=df8['passing_yards'], name=df8['name'],))
hover = HoverTool(
        tooltips=[("Player", "@name"),("40 Yard Dash Time", "$x{1.11}"),("Career Passing Yds", "@y"),])
p3 = figure(plot_width=600, plot_height=600, tools="pan,wheel_zoom,box_zoom,reset,resize", 
           title="QB: Passing Yards by 40 Yard Dash", x_axis_label ="40 Yard Dash Time", y_axis_label ="Career Passing Yards")
p3.add_tools(hover)
p3.circle('x', 'y', size=7, color='cyan', source=source)
tab3 = Panel(child=p3, title="QB")

tabs = Tabs(tabs=[ tab1, tab2, tab3 ])
show(tabs)

Loading BokehJS ...